In [1]:
import sys
import os
from pathlib import Path

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
sys.setrecursionlimit(5000)

In [3]:
from compute_ET_blue import (
    compute_et_blue,
    compute_volumetric_et_blue,
    postprocess_et_blue,
)

from utils import ee_utils, date_utils
from utils.ee_utils import back_to_float, back_to_int, export_image_to_asset

from typing import List, Tuple, Union

import ee
import geemap

In [4]:
ee.Initialize(project="thurgau-irrigation")

### Compute and export ET Blue raw (no postprocessing)

In [5]:
TIME_STEPS = 12
TIME_STEP_TYPE = "monthly"
YEARS = range(2022, 2023)

In [6]:
PATH_TO_AOI = "projects/thurgau-irrigation/assets/Zuerich/Zuerich_bound"

aoi_feature_collection = ee.FeatureCollection(PATH_TO_AOI)
aoi_geometry = aoi_feature_collection.geometry().simplify(500)

AOI = aoi_geometry.buffer(100)

In [7]:
ET_GREEN_COLLECTION = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Zuerich/ET_green_landsat_10m_jurisdiction_monthly_ZH_2022"
).map(lambda img: back_to_float(img, 100))

date_utils.print_collection_dates(ET_GREEN_COLLECTION)
ee_utils.print_value_ranges(ET_GREEN_COLLECTION, "ET_green")

Dates of images in the collection:
2022-01-01
2022-02-01
2022-03-01
2022-04-01
2022-05-01
2022-06-01
2022-07-01
2022-08-01
2022-09-01
2022-10-01
2022-11-01
2022-12-01
Image 1: Min = 3.30, Max = 10.39
Image 2: Min = 8.19, Max = 26.28
Image 3: Min = 12.11, Max = 33.15
Image 4: Min = 29.14, Max = 61.83
Image 5: Min = 54.94, Max = 91.79
Image 6: Min = 70.01, Max = 121.35
Image 7: Min = 65.62, Max = 129.55
Image 8: Min = 54.63, Max = 111.85
Image 9: Min = 31.55, Max = 81.51
Image 10: Min = 16.11, Max = 43.95
Image 11: Min = 0.00, Max = 0.00
Image 12: Min = 0.00, Max = 0.00


In [8]:
# path_to_WaPOR_ET_asset = "projects/thurgau-irrigation/assets/Thurgau/ET_WaPOR_10m_dekadal_test_refactor"


# WaPOR_ET_collections = ee_utils.merge_collections(
#     years=YEARS, asset_name=path_to_WaPOR_ET_asset
# )

ET_COLLECTION = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Zuerich/Landsat_ET_monthly_downscaled_10m_ZH_2022"
).map(lambda img: back_to_float(img, 100))

# ET_COLLECTION = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/ETlandsatmonthly"
# ).filterDate("2022-01-01", "2022-12-31")


# # Replace negative pixel values with 0 while retaining image properties
# def set_negative_to_zero(img):
#     return img.max(0).copyProperties(img, img.propertyNames())


# ET_COLLECTION = ET_COLLECTION.map(set_negative_to_zero)
# ET_COLLECTION = date_utils.set_to_first_of_month(ET_COLLECTION)


date_utils.print_collection_dates(ET_COLLECTION)
ee_utils.print_value_ranges(ET_COLLECTION, "downscaled")

Dates of images in the collection:
2022-01-01
2022-02-01
2022-03-01
2022-04-01
2022-05-01
2022-06-01
2022-07-01
2022-08-01
2022-09-01
2022-10-01
2022-11-01
2022-12-01
Image 1: Min = -8849.86, Max = 44657.47
Image 2: Min = -182.27, Max = 230.55
Image 3: Min = -539.40, Max = 430.08
Image 4: Min = -228.12, Max = 336.65
Image 5: Min = -316.47, Max = 710.06
Image 6: Min = -2295.27, Max = 944.03
Image 7: Min = -2996.71, Max = 614.55
Image 8: Min = -1812.83, Max = 743.15
Image 9: Min = -327.99, Max = 518.09
Image 10: Min = -27.86, Max = 2990.37
Image 11: Min = 0.00, Max = 0.00
Image 12: Min = 0.00, Max = 0.00


In [9]:
def process_and_export_et_blue_raw(
    wapor_et_list: ee.List,
    et_green_list: ee.List,
    year: int,
    aoi: ee.Geometry,
    time_steps: int = 12,
    time_step_type: str = "monthly",
) -> List[ee.batch.Task]:
    """
    Process and export ET blue raw images for a given year.

    Args:
        wapor_et_list (ee.List): List of WaPOR ET images for the year
        et_green_list (ee.List): List of ET green images for the year
        year (int): Year to process
        aoi (ee.Geometry): Area of interest to process
        time_steps (int): Number of time steps (default 12 for monthly)
        time_step_type (str): Type of time step ("monthly" or "dekadal")

    Returns:
        List[ee.batch.Task]: List of export tasks

    Raises:
        ValueError: If time_steps is not 12 or 36, or if time_step_type is invalid
    """
    if time_steps not in [12, 36]:
        raise ValueError("time_steps must be either 12 or 36")

    if time_step_type not in ["monthly", "dekadal"]:
        raise ValueError("time_step_type must be either 'monthly' or 'dekadal'")

    tasks = []

    for i in range(time_steps):
        # Get the ET images for current time step
        et_image = ee.Image(wapor_et_list.get(i))
        et_green = ee.Image(et_green_list.get(i))

        # Compute ET blue
        et_blue = compute_et_blue(et_image, et_green)
        et_blue = back_to_int(et_blue, 100)

        # Generate time step name based on type
        time_step_name = _get_time_step_name(i, time_step_type)

        # Generate export task
        task = _create_export_task(
            et_blue=et_blue,
            year=year,
            time_step_name=time_step_name,
            time_step_type=time_step_type,
            aoi=aoi,
        )

        tasks.append(task)

    return tasks


def _get_time_step_name(index: int, time_step_type: str) -> str:
    """
    Generate the time step name based on index and type.

    Args:
        index (int): Current time step index
        time_step_type (str): Type of time step ("monthly" or "dekadal")

    Returns:
        str: Formatted time step name
    """
    if time_step_type == "dekadal":
        month = index // 3 + 1
        dekad = index % 3 + 1
        return f"{month:02d}_D{dekad}"
    else:  # monthly
        month = index + 1
        return f"{month:02d}"


def _create_export_task(
    et_blue: ee.Image,
    year: int,
    time_step_name: str,
    time_step_type: str,
    aoi: ee.Geometry,
) -> ee.batch.Task:
    """
    Create an export task for an ET blue image.

    Args:
        et_blue (ee.Image): ET blue image to export
        year (int): Processing year
        time_step_name (str): Formatted time step name
        time_step_type (str): Type of time step
        aoi (ee.Geometry): Area of interest

    Returns:
        ee.batch.Task: Export task
    """
    task_name = f"ET_blue_raw_landsat_10m_ZH_{year}-{time_step_name}"

    asset_id = f"projects/thurgau-irrigation/assets/Zuerich/ET_blue_raw_landsat_10m_{time_step_type}_ZH_2022/{task_name}"

    return export_image_to_asset(
        image=et_blue,
        asset_id=asset_id,
        task_name=task_name,
        year=year,
        aoi=aoi,
        scale=10,
    )

In [10]:
# all_tasks = []

# for year in YEARS:

#     et_list = ET_COLLECTION.filterDate(f"{year}-01-01", f"{year}-12-31").toList(
#         TIME_STEPS
#     )

#     et_green_list = ET_GREEN_COLLECTION.filterDate(
#         f"{year}-01-01", f"{year}-12-31"
#     ).toList(TIME_STEPS)

#     # Process and export
#     tasks = process_and_export_et_blue_raw(
#         wapor_et_list=et_list,
#         et_green_list=et_green_list,
#         year=year,
#         aoi=AOI,
#         time_steps=TIME_STEPS,
#         time_step_type=TIME_STEP_TYPE,  # or "dekadal" as needed
#     )

#     print(f"Year {year} processing complete. Started {len(tasks)} tasks.")

#     all_tasks.extend(tasks)

# print(f"Total tasks: {len(all_tasks)}")

### Sanity check

In [14]:
et_blue_raw = ee.ImageCollection("projects/thurgau-irrigation/assets/Zuerich/ET_blue_raw_landsat_10m_monthly_ZH_2022").map(lambda img: back_to_float(img, 100))
et_blue_raw2 = ee.ImageCollection("projects/thurgau-irrigation/assets/Zuerich/ET_blue_raw_landsat_30m_monthly_ZH_2022").map(lambda img: back_to_float(img, 100))


# date_utils.print_collection_dates(et_blue_raw)
# ee_utils.print_value_ranges(et_blue_raw)

In [15]:
Map = geemap.Map()

et_blue_raw_list = et_blue_raw.toList(12)
et_blue_raw2_list = et_blue_raw2.toList(12)

image = ee.Image(et_blue_raw_list.get(5))
image2 = ee.Image(et_blue_raw2_list.get(5))


vis_params = {
    "bands" : ["ET_blue"], 
    "min" : -50,
    "max" : 50,
    "palette" : ["blue", "lightblue", "green", "yellow", "orange", "red"]
}

Map.center_object(AOI, 12)
Map.addLayer(image, vis_params, "ET blue raw")
Map.addLayer(image2, vis_params, "ET blue raw 30m")

Map

Map(center=[47.41511720684401, 8.655992127451274], controls=(WidgetControl(options=['position', 'transparent_b…